In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 49.5 MB/s 
     |████████████████████████████████| 6.6 MB 61.9 MB/s 
     |████████████████████████████████| 880 kB 43.4 MB/s 
     |████████████████████████████████| 77 kB 3.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e048ba21be2584a0d9c3674be16652676bad10051ad05dbe1ffe4d17c28e10a5
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 14.4 MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd /content/gdrive/Shareddrives/DL

Mounted at /content/gdrive/
/content/gdrive/Shareddrives/DL


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, RobertaForSequenceClassification, BertTokenizer
from transformers import PLBartTokenizer, PLBartForConditionalGeneration
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
plb_model =  PLBartForConditionalGeneration.from_pretrained("PLBART Model",max_position_embeddings=1500,ignore_mismatched_sizes=True).to(device)
plb_model.to(device)

PLBartForConditionalGeneration(
  (model): PLBartModel(
    (shared): Embedding(50005, 768, padding_idx=1)
    (encoder): PLBartEncoder(
      (embed_tokens): Embedding(50005, 768, padding_idx=1)
      (embed_positions): PLBartLearnedPositionalEmbedding(1502, 768)
      (layers): ModuleList(
        (0): PLBartEncoderLayer(
          (self_attn): PLBartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,)

In [5]:
tokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-base",src_lang="java", tgt_lang="java")

def PLBartTokenizerForSeq2SeqTest(x):
  return tokenizer.batch_encode_plus(x,max_length=750,truncation=True,padding='max_length',return_tensors="pt", add_special_tokens = True)

Downloading:   0%|          | 0.00/963k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783 [00:00<?, ?B/s]

In [8]:
tc2code = pd.read_json('./ResultValidation/data_testing.jsonl',lines=True)

In [9]:
tc2code.head()

,testCode,sourceCode
0,package com.example.util;\n\nimport com.exampl...,package com.example.util;\n\nimport com.exampl...
1,package com.example;\n\nimport org.junit.Test;...,package com.example;\n\nimport com.example.mod...
2,package com.example.web;\n\nimport com.example...,package com.example.web;\n\nimport com.example...
3,package com.example.service;\n\nimport com.exa...,package com.example.service;\n\nimport com.exa...
4,package com.practicalunittesting.chp03.money;\...,package com.practicalunittesting.chp03.money;\...


In [10]:
X_test, y_test = tc2code['testCode'], tc2code['sourceCode']


In [11]:
print(len(X_test.to_list()))

10


In [12]:
print(type(PLBartTokenizerForSeq2SeqTest(X_test.to_list()[1:2])))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [14]:
print(X_test.to_list()[3])

package com.example.service;

import com.example.model.ToDo;
import com.example.repository.ToDoRepository;
import org.junit.Before;
import org.junit.Test;
import org.junit.runner.RunWith;
import org.mockito.InjectMocks;
import org.mockito.Mock;
import org.mockito.MockitoAnnotations;
import org.springframework.test.context.junit4.SpringJUnit4ClassRunner;

import java.util.ArrayList;
import java.util.List;
import java.util.Optional;

import static org.junit.Assert.assertEquals;
import static org.mockito.Mockito.*;

@RunWith(SpringJUnit4ClassRunner.class)
public class ToDoServiceTest {

    @Mock
    private ToDoRepository toDoRepository;

    @InjectMocks
    private ToDoServiceImpl toDoService;

    @Before
    public void setup() {
        MockitoAnnotations.initMocks(this);
    }

    @Test
    public void testGetAllToDo() {
        List<ToDo> toDoList = new ArrayList<ToDo>();
        toDoList.add(new ToDo(1, "Todo Sample 1", true));
        toDoList.add(new ToDo(2, "Todo Sample 2", t

In [15]:
print(y_test.to_list()[3])

package com.example.service;

import com.example.model.ToDo;

import java.util.List;
import java.util.Optional;

public interface ToDoService {
    List<ToDo> getAllToDo();

    Optional<ToDo> getToDoById(long id);

    ToDo saveToDo(ToDo todo);

    void removeToDo(ToDo todo);
}



In [16]:
from torch.utils import data
class CodeTCDatasetTest(data.Dataset):
    def __init__(self, testTensor,expectedSourceCode,testCodeInput):
      self.testTensor=testTensor
      self.expectedSourceCode=expectedSourceCode
      self.testCodeInput=testCodeInput


    def __getitem__(self, index):
        return self.testTensor[index],self.expectedSourceCode[index],self.testCodeInput[index]


    def __len__(self):
        return len(self.testCodeInput)


def collate_fn(data):
    testTensorList,expectedSourceCode,testCodeInput = zip(*data)

    testTensors = torch.stack(testTensorList, 0)
      
    return testTensors, expectedSourceCode, testCodeInput

def get_test_loader(testTensor,expectedSourceCode,testCodeInput,batch_size):
    code2tctest = CodeTCDatasetTest(testTensor,expectedSourceCode,testCodeInput)
    data_loader = torch.utils.data.DataLoader(dataset=code2tctest, 
                                              batch_size=batch_size,
                                              collate_fn=collate_fn)
    return data_loader

In [17]:
X_test_tensors=PLBartTokenizerForSeq2SeqTest(X_test.to_list())
input_ids_test = X_test_tensors['input_ids']

In [18]:
print(len(input_ids_test))

10


In [19]:
batch_size=10
test_loader=get_test_loader(input_ids_test,y_test.to_list(),X_test.to_list(),batch_size)

In [20]:
print(type(X_test_tensors))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [21]:
def getPredictionsGreedy(inputs,model,device):
      greedy_output = model.generate(inputs.to(device), max_length=1500)
      sequences=[]
      for i in range(greedy_output.shape[0]):
        sequences.append(tokenizer.decode(greedy_output[i], skip_special_tokens=True))
      return sequences

In [22]:
def getPredictedTexts(dataloader_test,genAlgo,model,device,targIters=1,need_targ_iters=False):
    predictions=[]
    allExpectedprogs=[]
    allIpProgs=[]
    idx=0
    with torch.no_grad():
      for ip,estr,istr in dataloader_test:
          if idx>=targIters and need_targ_iters:
            break
          print(f"iter{idx} of {len(dataloader_test)}")
          prediction=genAlgo(ip,model,device)
          predictions=predictions+prediction
          expected=[x for x in estr]
          inputs=[x for x in istr]
          allExpectedprogs=allExpectedprogs+expected
          allIpProgs=allIpProgs+inputs
          idx+=1
    return allExpectedprogs,predictions,allIpProgs

In [23]:
allExpectedprogsGreedy,predictionsGreedy,ipTestCode=getPredictedTexts(test_loader,getPredictionsGreedy,plb_model,device)

iter0 of 1


In [24]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [25]:
print(allExpectedprogsGreedy[0])

package com.example.util;

import com.example.model.ToDo;

public class PayloadValidator {

    public static boolean validateCreatePayload(ToDo toDo) {
        return toDo.getId() <= 0;
    }

}



In [26]:
print(ipTestCode[0])

package com.example.util;

import com.example.model.ToDo;
import org.junit.Test;

import static org.junit.Assert.assertEquals;

public class PayloadValidatorTest {

    @Test
    public void validatePayLoad() {
        ToDo toDo = new ToDo(1, "Sample ToDo 1", true);
        assertEquals(false, PayloadValidator.validateCreatePayload(toDo));
    }

    @Test
    public void validateInvalidPayLoad() {
        ToDo toDo = new ToDo(0, "Sample ToDo 1", true);
        assertEquals(true, PayloadValidator.validateCreatePayload(toDo));
    }


}



In [27]:
print(predictionsGreedy[0])

.DoDo; import.text.Message; import.util.ArrayList; import.util.List; import.util.Validator; import com.mythoughtworks.xstream.annotations.XStreamException; import com.mythoughtworks.xstream.annotations.ValidationException; /** * * @author MyEclipse Hibernate framework */ public class ToDoValidator { public static void validateInvalidPayLoad(boolean validate, String message) { if (validate) { validateInvalidPayLoad(message); } } /** * * @param validate */ public static void validateInvalidPayLoad(boolean validate, String message) { if (validate) { validateInvalidPayLoad(message); } } /** * * @param validate */ public static void validateInvalidPayEquals(boolean validate, String message) { if (validate) { validateInvalidPayEquals(message); } } }


In [28]:
print(predictionsGreedy[1])

.test; import org.springframework.context.ApplicationTest; @Test public class DemoSpringTest extends Test { public void testLoad() { SpringApplicationContext = new SpringApplicationContext(); } }


In [29]:
print(predictionsGreedy[2])

.. import.util.ArrayList; import.util.Comparator; import.util.HashMap; import.util.List; import.util.Map; import.util.Set; import org.springframework.context.ApplicationContext; import org.springframework.context.ApplicationContextException; import org.springframework.context.support.ClassPathXmlApplicationContext; import org.springframework.context.support.ClassPathXmlApplicationContext; import org.springframework.context.support.ClassPathXmlApplicationContext; import org.springframework.context.support.ClassPathXmlApplicationContext; import org.springframework.context.support.ClassPathXmlApplicationContext; import org.springframework.context.support.ClassPathCacheManager; import org.springframework.context.support.bean.BeanPropertyBeanFactory; import org.springframework.context.support.bean.PathBean; import org.springframework.context.support.WebApplicationContextThreadLocal; import org.springframework.context.support.bean.RefreshBeanContextThreadLocal; import org.springframework.con

In [30]:
predictiondf_plb = pd.DataFrame(list(zip(allExpectedprogsGreedy, predictionsGreedy,ipTestCode)), columns =['Expected', 'predictions','ipUnitTest']) 

In [31]:
predictiondf_plb.to_json('ResultValidation/predictions_testing_plbart.jsonl',orient='records', lines=True)